# Initial Setup

In [1]:
# This is needed only for the purpose of the notebook
!pip install ipytest


[notice] A new release of pip is available: 24.2 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importing required libraries
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score

import pytest
import ipytest
ipytest.autoconfig()

In [3]:
# Getting the data
iris = datasets.load_iris()

In [4]:
# Simple setup in the data
iris_df = pd.DataFrame(iris.data, columns = iris.feature_names)
iris_df['target'] = iris.target

### Setting up the classes defined in the previous notebooks

In [5]:
class SimplePipeline:
    def __init__(self):
        self.frame = None
        # Each value is None when we instantiate the class
        self.X_train, self.X_test, self.y_train, self.Y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        """Loading the dataset, and make the train, test, split."""
        dataset = datasets.load_iris()
        
        # Removing the units (cm) from the headers
        self.feature_names = [fn[:-5] for fn in dataset.feature_names]
        self.frame = pd.DataFrame(dataset.data, columns=self.feature_names)
        self.frame['target'] = dataset.target
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[self.feature_names], self.frame.target, test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        """Execution method for running the pipeline several times."""
        self.load_dataset()
        self.train()

In [6]:
class PipelineWithFeatureEngineering(SimplePipeline):
    def __init__(self):
        # Calling the inherit method SimplePipeline __init__ first.
        super().__init__()
        
        # Standardizing the variables in the dataset.
        self.scaler = StandardScaler()
        # Training the pipeline
        self.scaler.fit(self.X_train)
    
    def apply_scaler(self):
        # Scaling training and testing data with mean 0 and variance 1.
        self.X_train = self.scaler.transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)
        
    def predict(self, input_data):
        # Applying the scaler before making the predictions.
        scaled_input_data = self.scaler.transform(input_data)
        return self.model.predict(scaled_input_data)
                  
    def run_pipeline(self):
        self.load_dataset()
        self.apply_scaler()
        self.train()

# Testing

We will check two different tests:
- Benchmark test: Comparing the accuracy of the model against a simple benchmark
- Differencial test: Comparing the accuracy of one model against the other

Let's first predict the most common class

In [7]:
iris_df['target'].value_counts()

target
0    50
1    50
2    50
Name: count, dtype: int64

In [8]:
@pytest.fixture
def pipelines():
    pipeline_v1 = SimplePipeline()
    pipeline_v2 = PipelineWithFeatureEngineering()
    pipeline_v1.run_pipeline()
    pipeline_v2.run_pipeline()
    return pipeline_v1, pipeline_v2

In [14]:
%%ipytest

def test_accuracy_higher_than_benchmark(pipelines):
    pipeline_v1, _ = pipelines
    
    # Initial Benchmark
    benchmark_predictions = [1.0] * len(pipeline_v1.y_test)
    benchmark_accuracy = accuracy_score(y_true=pipeline_v1.y_test, y_pred=benchmark_predictions)
    
    # Getting the accuracy of the model
    predictions = pipeline_v1.predict(pipeline_v1.X_test)
    actual_accuracy = accuracy_score(y_true=pipeline_v1.y_test, y_pred=predictions)
    
    print(f'Accuracy of model 1: {actual_accuracy}, Accuracy of Benchmark: {benchmark_accuracy}')
    
    # Comparing the accuracy of the first model against the benchmark
    assert actual_accuracy > benchmark_accuracy



.                                                                                            [100%]
======================================== warnings summary =========================================
notebooks/Testing/t_a8c348e3f24f48089dc21b6d75c144c8.py::test_accuracy_higher_than_benchmark
notebooks/Testing/t_a8c348e3f24f48089dc21b6d75c144c8.py::test_accuracy_higher_than_benchmark
  d:\Training\ITESM\MLOps_ITESM\practice3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
    warnings.warn(

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
1 passed, 2 warnings in 0.06s


In [11]:
%%ipytest

def test_accuracy_compared_to_previous_version(pipelines):
    pipeline_v1, pipeline_v2 = pipelines
    
    # Getting the accuracy of each version
    v1_accuracy = pipeline_v1.get_accuracy()
    v2_accuracy = pipeline_v2.get_accuracy()
    
    print(f'Accuracy of model 1: {v1_accuracy}')
    print(f'Accuracy of model 2: {v2_accuracy}')
    
    # Comparing the accuracy of the second model against the first one
    assert v2_accuracy >= v1_accuracy

F                                                                                            [100%]
============================================ FAILURES =============================================
___________________________ test_accuracy_compared_to_previous_version ____________________________

pipelines = (<__main__.SimplePipeline object at 0x0000025141675150>, <__main__.PipelineWithFeatureEngineering object at 0x000002514166C5D0>)

    def test_accuracy_compared_to_previous_version(pipelines):
        pipeline_v1, pipeline_v2 = pipelines
    
        # Getting the accuracy of each version
        v1_accuracy = pipeline_v1.get_accuracy()
        v2_accuracy = pipeline_v2.get_accuracy()
    
        print(f'Accuracy of model 1: {v1_accuracy}')
        print(f'Accuracy of model 2: {v2_accuracy}')
    
        # Comparing the accuracy of the second model against the first one
>       assert v2_accuracy >= v1_accuracy
E       assert 0.9591836734693877 >= 0.9693877551020408

C:\Users\